In [1]:
from VPython.NetWork import *

In [2]:
from PIL import Image

Input = Tensor(28, 28)
out = Convolution2D(filters=2, kernel=(3, 3), activation='relu', biasUsed=True)(Input)
out = MaxPooling2D((3,3))(out)
out = Convolution2D(filters=4, kernel=(3, 3), activation='relu')(out)
out = MaxPooling2D((3,3))(out)
out = Flatten()(out)
out = Dense(neurons=30, activation='logistic')(out)
softmaxOut = Dense(neurons=12, activation="softmax")(out)
optimizer = SGD(lr=0.1, decay=1.0, clipvalue=10)

model = Model(input=Input, output=softmaxOut)
model.compileLoss(Cross_Entropy())
model.compileRegular(L2Regularization(lamd=0.001))
model.compileOptimizer(optimizer)

In [3]:

class CustomImageDataset:
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = annotations_file
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        img1_path=os.path.join(self.img_dir,'1')
        self.n = len(os.listdir(img1_path))

    def __len__(self):
        return len(self.img_labels)*self.n

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir,str(int(idx/self.n )+ 1)+'/'+str(idx%self.n+1)+".bmp")
        with Image.open(img_path) as im:
            image = im.getdata()
        label = int(idx/self.n )+ 1
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [4]:
def transform(x):
    x = np.array(x).reshape(28,28)
    return x/255
def target_transform(x):
    res = np.zeros((12,1),dtype=np.float16)
    res[int(x-1)][0] = 1.0
    return res

annotations='博学笃志切问近思自由无用'
annotations=list(annotations)
print(annotations)
imageSet = CustomImageDataset(annotations,'./train/',transform,target_transform)
image, label=imageSet[2212]
image

['博', '学', '笃', '志', '切', '问', '近', '思', '自', '由', '无', '用']


array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0

In [5]:
def fit(model:Model,Train,Test, epoch:int =1,batch_size: int = 1, log: bool = True):
    nn=len(Train)
    input = 0
    sumloss = 0
    loss = 0
    error = False
    lr = model.optimizer.lr
    iteration = int(nn/batch_size)
    print("iteration",iteration)
    lastLoss = 0
    allLoss = 0.0
    worse = 0
    TrainAcc = 0
    TestAcc = 0
    for i in range(epoch):
        TrainAcc = 0
        TestAcc = 0
        print(i)
        testIndex = list(range(nn))
        np.random.shuffle(testIndex)
        lastLoss = allLoss
        allLoss = 0.0
        for j in range(iteration):

            sumloss=0.0
            for k in range(batch_size):
                testIndex0 = testIndex[j*batch_size+k]
                image,label = Train[testIndex0]
                input = model.predict(image)
                derivation, loss = model.lossFunc(f=input, res=label)
                if np.isnan(loss):
                    error = True
                    print("Error")
                    break
                for item in model.layer[-1::-1]:
                    derivation = item.feedBackward(derivation)
                    derivation = np.array(derivation)
                if np.argmax(input) == np.argmax(label):
                    TrainAcc = TrainAcc + 1
                
                sumloss = sumloss + loss
                allLoss = allLoss + loss
            for item in model.layer:
                if error:
                    item.abort()
                    print("Error")
                    return
                else:
                    item.parameterUpdate(batch_size, lr, model.regularization)
            # if xtest is not None and ytest is not None and step is not None:
        for j in range(len(Test)):
            image,label = Test[j]
            input = model.predict(image)
            if np.argmax(input) == np.argmax(label):
                TestAcc = TestAcc +1
        if allLoss > lastLoss:
            worse = worse+1
        else:
            worse  = 0
        if worse > 3:
            lr = max(lr*0.9,0.001)
        if log:
            print("epoch=%d,loss = %.8f,trainAcc=%.8f,testAcc = %.8f" % (i, allLoss/iteration/batch_size,TrainAcc/nn,TestAcc/len(Test)))
        lr = max(lr * model.optimizer.decay,0.001)
mylist = list(range(620))
rate = 0.1

Train = []
Test = []
for i in range(12):
    np.random.shuffle(mylist)
    for j in range(620):
        if j < 620*rate:
            Test.append(imageSet[i*620+mylist[j]])
        else:
            Train.append(imageSet[i*620+mylist[j]])

print(len(Train))
print(len(Test))
fit(model,Train,Test,100,24)

6696
744
iteration 279
0
epoch=0,loss = 2.49496019,trainAcc=0.08542413,testAcc = 0.08333333
1
epoch=1,loss = 2.48217192,trainAcc=0.09916368,testAcc = 0.14247312
2
epoch=2,loss = 2.23579812,trainAcc=0.25358423,testAcc = 0.37231183
3
epoch=3,loss = 1.65353821,trainAcc=0.44578853,testAcc = 0.47983871
4
epoch=4,loss = 1.40106901,trainAcc=0.51702509,testAcc = 0.52688172
5
epoch=5,loss = 1.26236771,trainAcc=0.57750896,testAcc = 0.53763441
6
epoch=6,loss = 1.17509182,trainAcc=0.60946834,testAcc = 0.60887097
7
epoch=7,loss = 1.11887227,trainAcc=0.63172043,testAcc = 0.64247312
8
epoch=8,loss = 1.06651325,trainAcc=0.64799881,testAcc = 0.63440860
9
epoch=9,loss = 1.02152694,trainAcc=0.66442652,testAcc = 0.64516129
10
epoch=10,loss = 0.98953980,trainAcc=0.66816010,testAcc = 0.66935484
11
epoch=11,loss = 0.94667056,trainAcc=0.69429510,testAcc = 0.64784946
12
epoch=12,loss = 0.90869347,trainAcc=0.70922939,testAcc = 0.69354839
13
epoch=13,loss = 0.89250947,trainAcc=0.70758662,testAcc = 0.71505376
14


In [ ]:
model.logModel("./model/character1.hdf5")
def evaluate(model:Model,Train):
    nn=len(Train)
    acc = 0
    for i in range(nn):
        image, label = Train[i]
        input = model.predict(image)
        if np.argmax(input) == np.argmax(label):
            acc += 1
    acc = acc/nn
    print("acc",acc)
    
evaluate(model,Test)

